In [1]:
import numpy as np 
import pandas as pd
import simpy
import random

In [2]:
df1 = pd.read_excel('profiles.xlsx','intraday',index_col=0)
df2 = pd.read_excel('profiles.xlsx','intraweek')
df3 = pd.read_excel('profiles.xlsx','weeklyForecast', index_col = 0)

# CORE METRICS TO BE ACHIVED 
PCA = 0.9
SERVICE_TIME = 20
SLA = 0.8
OCCTARGET = 0.85
SHRINK = 0.25

In [3]:
""" erlang functions in python 
    For first attempt at online planning tool 
"""

import math
import numpy

def agents_req(calls, reporting_period,aht,service_level, service_time):
    """ calculates the minimum agents required to achieve service level

    """
    still_validating = False

    if reporting_period < 5:
        still_validating = True

    if reporting_period > 1500:
        still_validating = True

    if aht < 1:
        still_validating = True

    if aht > 30000:
        still_validating = True

    if service_level < 0.00001:
        still_validating = True

    if service_level > 0.9998:
        still_validating = True

    if service_time < 1:
        still_validating = True

    if service_time > 30000:
        still_validating = True
    
    if still_validating == True:
        return None

    intensity = (calls/(reporting_period * 60)) * aht

    agents = int(intensity)
    #print("agents = " + str(agents))
    
    while servicelevel(calls, reporting_period, aht, service_time, agents) < service_level:
        #print("------")
        #print(" from while loop in Agents_Req function")
        #print("Agents = " + str(agents))
        #print("serviceLevel = " + str(servicelevel(calls, reporting_period, aht, service_time, agents)))
        #print(("Service_level target = " + str(service_level)))
        agents = agents + 1
    
    return agents


def servicelevel(calls, reporting_period, aht, service_time, agents):

    still_validating = False

    if reporting_period < 5:
        still_validating = True
    if reporting_period > 1500:
        still_validating = True
    if aht < 1:
        still_validating = True
    if aht > 30000:
        still_validating = True
    if service_time < 1:
        still_validating = True
    if service_time > 30000:
        still_validating = True
    
    if still_validating == True:
        return None

    intensity = (calls/ (reporting_period * 60)) * aht

    #print("Intensity = " + str(intensity))
    #print("calls = " + str(calls))
    #print(" Reporting Period = " + str(reporting_period))

    #print("------- Service Level calcs -------")
    #print("EXP function - " + str(numpy.exp(-(agents - intensity) * service_time/aht)))
    #print("EXP Functio to go test = || (-("+ str(agents) + " - " + str(intensity) + ") * " + str(service_time) + " / " + str(aht))

    #print("Prob call waits = " + str(prob_calls_waits(calls, reporting_period, aht, agents)))
    servicelevel = 1 - (prob_calls_waits(calls, reporting_period, aht, agents) * numpy.exp(-(agents - intensity) * service_time/aht))


    if servicelevel > 1:
        return 1

    if servicelevel < 0:
        return 0

    return servicelevel


def prob_calls_waits(calls, reporting_period, aht, agents):

    intensity = (calls / (reporting_period * 60)) * aht

    occupancy = intensity / agents

    a_n = 1
    sum_a_k = 0

    #print("sum ak agents = " + str(agents))

    for k in range(agents,1,-1):
        #print("K = " + str(k))
        a_k = a_n * k / intensity
        sum_a_k += + a_k
        a_n = a_k

    #print("Sum a_k = " + str(sum_a_k))
    
    probcallwaits = 1 / (1 + ((1-occupancy) * sum_a_k))

    #print("prob call waits in its function = " + str(probcallwaits))

    if probcallwaits > 1:
       return 1

    if probcallwaits < 0:
        return 0

    return probcallwaits
        

def utilisation(calls, reporting_period, aht, agents):

    intensity = (calls/(reporting_period * 60) * aht)

    util = intensity / agents

    if util < 0:
        return 0
    if util >1:
        return 1
    
    return util

In [4]:
# work out how to create the weekly dataframe so that we can work off of that


# get the weeks here, cycle from top level down is my thinking

dfWeek = pd.DataFrame(columns=('interval','monday','tuesday','wednesday','thursday','friday'))

for index, row in df3.iterrows():
   week = index.date()
   callFcast = row['calls_offered']
   mondayVol = row['calls_offered'] * df2.iloc[0]['Monday']
   tuesdayVol = row['calls_offered'] * df2.iloc[0]['Tuesday']
   wednesdayVol = row['calls_offered'] * df2.iloc[0]['Wednesday']
   thursdayVol = row['calls_offered'] * df2.iloc[0]['Thursday']
   fridaVol = row['calls_offered'] * df2.iloc[0]['Friday']
   ahtFcast = row['calls_AHT']

   for index, row in df1.iterrows():
    new_row = {
        'week':week,
        'interval' : index, 
        'monday':mondayVol * df1.loc[index]['Monday'],
        'tuesday':tuesdayVol * df1.loc[index]['Tuesday'],
        'wednesday':wednesdayVol * df1.loc[index]['Wednesday'],
        'thursday':thursdayVol * df1.loc[index]['Thursday'],
        'friday':fridaVol * df1.loc[index]['Friday'],
        'aht':ahtFcast,
        'monErlang':agents_req(mondayVol * df1.loc[index]['Monday'],15,ahtFcast,SLA,SERVICE_TIME),
        'tueErlang':agents_req(tuesdayVol * df1.loc[index]['Tuesday'],15,ahtFcast,SLA,SERVICE_TIME),
        'wedErlang':agents_req(wednesdayVol * df1.loc[index]['Wednesday'],15,ahtFcast,SLA,SERVICE_TIME),
        'thuErlang':agents_req(thursdayVol * df1.loc[index]['Thursday'],15,ahtFcast,SLA,SERVICE_TIME),
        'friErlang':agents_req(fridaVol * df1.loc[index]['Friday'],15,ahtFcast,SLA,SERVICE_TIME)

        }

    dfNewRow = pd.DataFrame.from_records([new_row])
    dfWeek = pd.concat([dfWeek, dfNewRow])
   
   print ('week:%s || calls: %d || aht:  %r' % (week, callFcast, ahtFcast))


week:2022-06-06 || calls: 100000 || aht:  450
week:2022-06-13 || calls: 110000 || aht:  450
week:2022-06-20 || calls: 120000 || aht:  450


<h1> Below is the Simulation element of the process</h1>

In [7]:
# This the car, in the new model it will be a call
def call(env, callID, agents, arrival_time, call_duration):

    
    #Simulate driving to the station
    yield env.timeout(arrival_time)
    callWaiting = env.now

    print('%s arrived at %d' % (callID, env.now))
    with agents.request() as req:
        yield req

        #charge the battery 
        print('%s started chatting at %s : was waiting for %r' % (callID, env.now, env.now - callWaiting))
        print(agents.count)
        yield env.timeout(call_duration)
        print(agents.count)
        print('%s ended the call with the agent at %s' % (callID, env.now))

env = simpy.Environment() # This is the sim enviroment that will be live
agents = simpy.Resource(env, capacity=500) # specify the resource, this will be agent


#I need to order the calls of the day into the correct order of arrival and then feed them through in sequential order


#Since car is call, then the i will be number of calls that happen instantly, this will be random over the interval period?

callID = 0
for i in range(10000):
    arrival_time = random.randrange(0,900)
    callID = callID + 1
    callAHT = 450 * (1-random.randrange(-10,10)/100)
    env.process(call(env, callID, agents, arrival_time, callAHT))

# this will kick off the process
env.run()


658 arrived at 0
2087 arrived at 0
3578 arrived at 0
3971 arrived at 0
5468 arrived at 0
6310 arrived at 0
6322 arrived at 0
6739 arrived at 0
7400 arrived at 0
7453 arrived at 0
8837 arrived at 0
9285 arrived at 0
9382 arrived at 0
658 started chatting at 0 : was waiting for 0
13
2087 started chatting at 0 : was waiting for 0
13
3578 started chatting at 0 : was waiting for 0
13
3971 started chatting at 0 : was waiting for 0
13
5468 started chatting at 0 : was waiting for 0
13
6310 started chatting at 0 : was waiting for 0
13
6322 started chatting at 0 : was waiting for 0
13
6739 started chatting at 0 : was waiting for 0
13
7400 started chatting at 0 : was waiting for 0
13
7453 started chatting at 0 : was waiting for 0
13
8837 started chatting at 0 : was waiting for 0
13
9285 started chatting at 0 : was waiting for 0
13
9382 started chatting at 0 : was waiting for 0
13
269 arrived at 1
1614 arrived at 1
4049 arrived at 1
4397 arrived at 1
4712 arrived at 1
5346 arrived at 1
6755 arrive